Описание проекта.

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error, make_scorer
from collections import defaultdict

Запишем некоторые константы.

In [2]:
oil_revenue = 450000 # доход за тысячу баррелей сырья, в данных объем запасов указан в тыс.баррелей
wells_count = 500
wells_count_best = 200
budget_development = 10000000000
risk = 0.025
state=np.random.RandomState(12345)

Загрузим данные.

In [3]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
geo_data_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


Нам даны три региона, каждый регион в отдельном файле. Описание столбцов:
•	id — уникальный идентификатор скважины;
•	f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
•	product — объём запасов в скважине (тыс. баррелей).

Напишем функцию для получения информации о данных.

In [5]:
def info_dataset(dataset):
    print('Общая информация о столбцах регирна')
    print(dataset.info())
    print()
    print('Информация о зависимостях в столбцах')
    print(dataset.corr())
    print()
    print('Описательная статистика данных')
    print(dataset.describe())
    print()
    print('Количество повторяющихся строк')
    print(dataset.duplicated().sum())
    print()
    print('Количество скважин с 0-ми запасами нефти')
    print(dataset[dataset['product'] == 0]['product'].count())

## Первый регион

In [6]:
info_dataset(geo_data_0)

Общая информация о столбцах регирна
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Информация о зависимостях в столбцах
               f0        f1        f2   product
f0       1.000000 -0.440723 -0.003153  0.143536
f1      -0.440723  1.000000  0.001724 -0.192356
f2      -0.003153  0.001724  1.000000  0.483663
product  0.143536 -0.192356  0.483663  1.000000

Описательная статистика данных
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328

Пропущенных значений и дубликатов нет. Сильная зависимость количества нефти от параметра f2 - 48%. Менее от f1 и f0 - 19% и 14% соответственно. Есть одна скважина с 0 запасами нефти. Возможно это ошибка или так и есть на самом деле, но так как всего одна скважина, скорее всего она пустая/сухая.

## Второй регион

In [7]:
info_dataset(geo_data_1)

Общая информация о столбцах регирна
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Информация о зависимостях в столбцах
               f0        f1        f2   product
f0       1.000000  0.182287 -0.001777 -0.030491
f1       0.182287  1.000000 -0.002595 -0.010155
f2      -0.001777 -0.002595  1.000000  0.999397
product -0.030491 -0.010155  0.999397  1.000000

Описательная статистика данных
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144

Пропущенных значений и дубликатов нет. Очень сильная зависимость количества нефти от параметра f2 - 99,9%. Очень низкая зависимость от f1 и f0 - 1% и 3% соответственно. Большое количество скважин с 0 запасами нефти - 8235 шт. Возможно это ошибка или сбой. Я думаю, что стоит запросить разъяснения у заказчика. По количеству нефти второй регион сильно уступает первому - медиана второго региона по запасам 57, тогда как у первого - 92.

## Третий регион

In [8]:
info_dataset(geo_data_2)

Общая информация о столбцах регирна
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

Информация о зависимостях в столбцах
               f0        f1        f2   product
f0       1.000000  0.000528 -0.000448 -0.001987
f1       0.000528  1.000000  0.000779 -0.001012
f2      -0.000448  0.000779  1.000000  0.445871
product -0.001987 -0.001012  0.445871  1.000000

Описательная статистика данных
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.002023      -0.002081       2.495128      95.000000
std         1.732045       1.730417       3.473445      44.749921
min        -8.760004      -7.084020     -11.970335

Пропущенных значений и дубликатов нет. Сильная зависимость количества нефти от параметра f2 - 44,6%. Очень низкая от f1 и f0 - менее 1%. Есть одна скважина с 0 запасами нефти. Так как всего одна скважина, скорее всего она пустая/сухая. Запасы нефти сравнимы с первым регионом.

Посчитаем в денежном выражении запасы нефти в каждом регионе.

In [9]:
print('Регион №1', geo_data_0['product'].sum() * oil_revenue, 'рублей')
print('Регион №2', geo_data_1['product'].sum() * oil_revenue, 'рублей')
print('Регион №3', geo_data_2['product'].sum() * oil_revenue, 'рублей')

Регион №1 4162500000000.001 рублей
Регион №2 3097125000000.001 рублей
Регион №3 4275000000000.0015 рублей


Предварительно можно сказать, что доход с регионов 1 и 3 будет на 33% выше, чем с региона 2, так как в этих регионах сосредоточены довольно большие запасы нефти.

# Обучение и проверка модели

Разобьем данные на выборки, обучим модели и посчитаем метрики. Для этого напишем функцию.

In [10]:
results_valid = []
def model_check(dataset):
    features = dataset.drop(['id', 'product'], axis=1)
    target = dataset['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                 target, 
                                                                 train_size=0.75, 
                                                                 test_size=0.25, 
                                                                 random_state=state)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    
    MSE = mean_squared_error(target_valid, predictions)
    RMSE = MSE ** 0.5
    mean = predictions.mean()
    
    results_valid.append({'features_train': features_train, 
                          'features_valid': features_valid, 
                          'target_train': target_train, 
                          'target_valid': target_valid,
                          'predictions': pd.Series(predictions, index=target_valid.index),
                          'RMSE': RMSE,
                          'mean': mean})
    print('features_train:', features_train.shape) 
    print('features_valid:', features_valid.shape)

In [11]:
model_check(geo_data_0)
model_check(geo_data_1)
model_check(geo_data_2)

features_train: (75000, 3)
features_valid: (25000, 3)
features_train: (75000, 3)
features_valid: (25000, 3)
features_train: (75000, 3)
features_valid: (25000, 3)


Разделение произошло правильно. Посмотрим метрики.

In [12]:
cols = ['RMSE', 'mean']
pd.DataFrame(results_valid)[cols]

,RMSE,mean
0,37.579422,92.592568
1,0.889737,68.769951
2,39.958042,95.087528


Не смотря на то, что средние запасы нефти в скважинах первого и третьего регионов намного превышают средние запасы нефти во втором регионе, корень из средней квалдратичной ошибки очень высокий для первого и третьего регионов. А во втором регионе он ошибка почти нулевая.

# Подготовка к расчёту прибыли

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

Бюджет 10 миллиардов рублей, из 500 скважин только 200 пойдут в разработку. Получается, что на одну скважину мы потратим 50 миллионов рублей.

In [13]:
budget_per_well = budget_development / wells_count_best
print('Бюджет на одну скважину:', budget_per_well, 'рублей')

Бюджет на одну скважину: 50000000.0 рублей


Чтобы получить доход в 50 миллионов рублей нужно получить

In [14]:
break_even_barrels = budget_per_well / oil_revenue
print('Для точки безубыточности нужно добыть:', break_even_barrels, 'баррелей с каждой из 200 скважин')

Для точки безубыточности нужно добыть: 111.11111111111111 баррелей с каждой из 200 скважин


Получается, что с каждой скважины нужно добыть 112 тысяч баррелей нефти. Это намного меньше среднего запаса во втором регионе и на 20 тысяч баррелей меньше, чем в первом и третьем регионах.

# Расчёт прибыли и рисков 

Напишем функцию для расчета прибыли.

In [15]:
def revenue(target, probabilities):
    probs_best = probabilities.sort_values(ascending = False).head(wells_count_best).index
    income = target[probs_best].sum() * oil_revenue
    revenue_last = income - budget_development
    return revenue_last

In [16]:
bootstrap_results = []

for index, dictionary in enumerate(results_valid):
        
    target_valid = dictionary.get('target_valid')
    predictions = pd.Series(dictionary.get('predictions'), index=target_valid.index)
    
    values = []
    
    for i in range(1000):
        
        target_subsample = target_valid.sample(n=wells_count, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        
        values.append(revenue(target_subsample, probs_subsample))
    
    values = pd.Series(values)
    
    mean = values.mean() / 10e6
    lower = values.quantile(risk) / 10e6
    upper = values.quantile(1-risk) / 10e6
    confidence_interval = (lower, upper)
    risk_of_loss = (values < 0).sum() / values.count() *100
    
    bootstrap_results.append({'values': values,
                              'risk_of_loss': risk_of_loss,
                              'mean_revenue': mean, 
                              'confidence_interval': confidence_interval})

In [17]:
bootstrap_res = pd.DataFrame(bootstrap_results)
bootstrap_res.loc[:, bootstrap_res.columns != 'values']

,risk_of_loss,mean_revenue,confidence_interval
0,1.6,60.604362,"(4.480034210458375, 124.01965183875392)"
1,0.3,66.165661,"(18.00958570171501, 118.34926804174238)"
2,3.4,56.188077,"(-2.0425468812190197, 119.35085755051787)"


Исходя из полученных данных следует, что лучше всего для разработки выбрать регион №2, так как средний размер дохода тут выше и риски, получить отрицательную прибыль, минимальные.